# WORC Tutorial: Simple

Welcome to the tutorial of WORC: a Workflow for Optimal Radiomics Classification! It will provide you with basis knowledge and practical skills on how to run the WORC. For advanced topics and WORCflows, please see the other notebooks provided with this tutorial. For installation details, see the ReadMe.md provided with this tutorial.


This tutorial interacts with  WORC through SimpleWORC and is especially suitable for first time usage.

In [1]:
# impor neccesary packages
from WORC import SimpleWORC
import os

# These packages are only used in analysing the results
import pandas as pd
import json
import fastr
import glob

# If you don't want to use your own data, we use the following example set,
# see also the next code block in this example.
from WORC.exampledata.datadownloader import download_HeadAndNeck

# Define the folder this script is in, so we can easily find the example data
script_path = os.getcwd()

[WARNING]  __init__:0078 >> Not running in a production installation (branch "develop" from source code)
[WARNING]  warnings:0110 >> /home/martijn/Documents/WORC3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)

[WARNING]  warnings:0110 >> /home/martijn/Documents/WORC3/lib/python3.7/site-packages/phasepack/tools.py:14: UserWarning: 
Module 'pyfftw' (FFTW Python bindings) could not be imported. To install it, try
running 'pip install pyfftw' from the terminal. Falling back on the slower
'fftpack' module for 2D Fourier transforms.
  'fftpack' module for 2D Fourier transforms.""")





---------------------------------------------------------------------------
Input
---------------------------------------------------------------------------
The minimal inputs to WORC are:
  - Images
  - Segmentations
  - Labels

In SimpleWORC, we assume you have a folder "datadir", in which there is a
folder for each patient, where in each folder there is a image.nii.gz and a mask.nii.gz:
          Datadir
              Patient_001
                  image.nii.gz
                  mask.nii.gz
              Patient_002
                  image.nii.gz
                  mask.nii.gz
              ...


You can skip this part if you use your own data.
In the example, We will use open source data from the online XNAT platform
at https://xnat.bmia.nl/data/archive/projects/stwstrategyhn1. This dataset
consists of CT scans of patients with Head and Neck tumors. 

In [2]:
# Download a subset of 20 patients in this folder. You can change these if you want.
nsubjects = 20  # use "all" if you want to download all patients.
data_path = os.path.join(script_path, 'Data')
download_HeadAndNeck(datafolder=data_path, nsubjects=nsubjects)

Working on subject 1/137
resource is NIFTI


183.6 KiB |#                                                      |   4.4 MiB/s


resource is NIFTI


 21.4 MiB |           #                                           |  19.3 MiB/s


Working on subject 2/137
resource is NIFTI


...

Working on subject 20/137
resource is NIFTI


 34.4 KiB |#                                                      |   3.9 MiB/s


resource is NIFTI


 19.5 MiB |       #                                               |  27.3 MiB/s


Done downloading!


Define the inputs of our network

In [3]:
# Identify our data structure: change the fields below accordingly
# if you use your own data.
imagedatadir = os.path.join(data_path, 'stwstrategyhn1')
image_file_name = 'image.nii.gz'
segmentation_file_name = 'mask.nii.gz'

# File in which the labels (i.e. outcome you want to predict) is stated
# Again, change this accordingly if you use your own data.
label_file = os.path.join(data_path, 'Examplefiles', 'pinfo_HN.csv')

# Name of the label you want to predict
label_name = 'imaginary_label_1'

# Determine whether we want to do a coarse quick experiment, or a full lengthy
# one. Again, change this accordingly if you use your own data.
coarse = True

# Give your experiment a name
experiment_name = 'Example_STWStrategyHN'

# Instead of the default tempdir, let's but the temporary output in a subfolder
# in the same folder as this script
tmpdir = os.path.join(script_path, 'WORC_' + experiment_name)


---------------------------------------------------------------------------
The actual experiment
---------------------------------------------------------------------------

NOTE:  Precomputed features can be used instead of images and masks
by instead using ``I.features_from_this_directory()`` in a similar fashion to below. 

In [6]:
# Create a WORC object
experiment = SimpleWORC(experiment_name)

# Set the input data according to the variables we defined earlier
experiment.images_from_this_directory(imagedatadir,
                             image_file_name=image_file_name)
experiment.segmentations_from_this_directory(imagedatadir,
                                    segmentation_file_name=segmentation_file_name)
experiment.labels_from_this_file(label_file)
experiment.predict_labels([label_name])

# Use the standard workflow for binary classification
experiment.binary_classification(coarse=coarse)

# Set the temporary directory
experiment.set_tmpdir(tmpdir)



BigrClusterDetector detected False.
CartesiusClusterDetector detected False.
BigrClusterDetector detected False.
CartesiusClusterDetector detected False.


In [7]:
# Run the experiment!
experiment.execute()

DebugDetector detected False.
[INFO] networkrun:0517 >> ####################################
[INFO] networkrun:0518 >> #     network execution STARTED    #
[INFO] networkrun:0519 >> ####################################
[INFO] networkrun:0544 >> Running network via /home/martijn/Documents/WORC3/src/fastr/fastr/api/__init__.py (last modified Wed Oct 16 16:42:02 2019)
[INFO] networkrun:0545 >> FASTR loaded from /home/martijn/Documents/WORC3/src/fastr/fastr
[INFO] networkrun:0561 >> Network run tmpdir: /home/martijn/git/WORCTutorial/WORC_Example_STWStrategyHN
[INFO] networkchunker:0146 >> Adding classification to candidates (blocking False)
[INFO] networkchunker:0146 >> Adding performance to candidates (blocking False)
[INFO] networkchunker:0146 >> Adding features_train_CT_0 to candidates (blocking False)
[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/config_classification_source s

[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/segmentations_train_CT_0 sample id <SampleId ('HN1331',)>, index <SampleIndex (9)>
[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/segmentations_train_CT_0 sample id <SampleId ('HN1339',)>, index <SampleIndex (10)>
[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/segmentations_train_CT_0 sample id <SampleId ('HN1342',)>, index <SampleIndex (11)>
[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/segmentations_train_CT_0 sample id <SampleId ('HN1372',)>, index <SampleIndex (12)>
[INFO]   noderun:0576 >> Creating job

[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/convert_seg_train_CT_0 sample id <SampleId ('HN1088',)>, index <SampleIndex (2)>
[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/convert_seg_train_CT_0 sample id <SampleId ('HN1146',)>, index <SampleIndex (3)>
[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/convert_seg_train_CT_0 sample id <SampleId ('HN1159',)>, index <SampleIndex (4)>
[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/convert_seg_train_CT_0 sample id <SampleId ('HN1192',)>, index <SampleIndex (5)>
[INFO]   noderun:0576 >> Creating job for node f

[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/preprocessing_train_CT_0 sample id <SampleId ('HN1524',)>, index <SampleIndex (16)>
[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/preprocessing_train_CT_0 sample id <SampleId ('HN1554',)>, index <SampleIndex (17)>
[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/preprocessing_train_CT_0 sample id <SampleId ('HN1560',)>, index <SampleIndex (18)>
[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/preprocessing_train_CT_0 sample id <SampleId ('HN1748',)>, index <SampleIndex (19)>
[INFO]   noderun:0470 >> Generating 

[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/features_train_CT_0 sample id <SampleId ('HN1260',)>, index <SampleIndex (7)>
[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/features_train_CT_0 sample id <SampleId ('HN1323',)>, index <SampleIndex (8)>
[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/features_train_CT_0 sample id <SampleId ('HN1331',)>, index <SampleIndex (9)>
[INFO]   noderun:0576 >> Creating job for node fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/features_train_CT_0 sample id <SampleId ('HN1339',)>, index <SampleIndex (10)>
[INFO]   noderun:0576 >> Creating job for node fastr:///net

[INFO] networkrun:0765 >> Finished job WORC_Example_STWStrategyHN___features_train_CT_0___HN1146___0 with status JobState.cancelled
[INFO] networkrun:0765 >> Finished job WORC_Example_STWStrategyHN___features_train_CT_0___HN1088___0 with status JobState.cancelled
[INFO] networkrun:0765 >> Finished job WORC_Example_STWStrategyHN___features_train_CT_0___HN1077___0 with status JobState.cancelled
[INFO] networkrun:0765 >> Finished job WORC_Example_STWStrategyHN___features_train_CT_0___HN1004___0 with status JobState.cancelled
[INFO] networkrun:0765 >> Finished job WORC_Example_STWStrategyHN___calcfeatures_train_predict_CalcFeatures_1_0_CT_0___HN1748 with status JobState.cancelled
[ERROR]   noderun:0508 >> Could not find required data for fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/calcfeatures_train_predict_CalcFeatures_1_0_CT_0/outputs/features in {}!
[INFO] networkrun:0765 >> Finished job WORC_Example_STWStrategyHN___calcf

[ERROR]   noderun:0508 >> Could not find required data for fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/preprocessing_train_CT_0/outputs/image in {}!
[INFO] networkrun:0765 >> Finished job WORC_Example_STWStrategyHN___preprocessing_train_CT_0___HN1560 with status JobState.cancelled
[ERROR]   noderun:0508 >> Could not find required data for fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/preprocessing_train_CT_0/outputs/image in {}!
[INFO] networkrun:0765 >> Finished job WORC_Example_STWStrategyHN___preprocessing_train_CT_0___HN1554 with status JobState.cancelled
[ERROR]   noderun:0508 >> Could not find required data for fastr:///networks/WORC_Example_STWStrategyHN/0.0/runs/WORC_Example_STWStrategyHN_2019-10-16T17-50-17/nodelist/preprocessing_train_CT_0/outputs/image in {}!
[INFO] networkrun:0765 >> Finished job WORC_Example_STWStrategyHN___preprocessing_train_C

**NOTE:**  Precomputed features can be used instead of images and masks by instead using ``I.features_from_this_directory()`` in a similar fashion.

---------------------------------------------------------------------------
Analysis of results
---------------------------------------------------------------------------

There are two main outputs: the features for each patient/object, and the overall
performance. These are stored as .hdf5 and .json files, respectively. By
default, they are saved in the so-called "fastr output mount", in a subfolder
named after your experiment name.

In [ ]:
# Locate output folder
outputfolder = fastr.config.mounts['output']
experiment_folder = os.path.join(outputfolder, 'WORC_' + experiment_name)

print(f"Your output is stored in {experiment_folder}.")

# Read the features for the first patient
# NOTE: we use the glob package for scanning a folder to find specific files
feature_files = glob.glob(os.path.join(experiment_folder,
                                       'Features',
                                       'features_*.hdf5'))
featurefile_p1 = feature_files[0]
features_p1 = pd.read_hdf(featurefile_p1)

# Read the overall peformance
performance_file = os.path.join(experiment_folder, 'performance_all_0.json')
with open(performance_file, 'r') as fp:
    performance = json.load(fp)

# Print the feature values and names
print("Feature values:")
for v, l in zip(features_p1.feature_values, features_p1.feature_labels):
    print(f"\t {l} : {v}.")

# Print the output performance
print("\n Performance:")
stats = performance['Statistics']
del stats['Percentages']  # Omitted for brevity
for k, v in stats.items():
    print(f"\t {k} {v}.")

**NOTE:** the performance is probably horrible, which is expected as we ran
the experiment on coarse settings. These settings are recommended to only
use for testing: see also below.


---------------------------------------------------------------------------
Tips and Tricks
---------------------------------------------------------------------------

For tips and tricks on running a full experiment instead of this simple
example, adding more evaluation options, debuggin a crashed network etcetera,
please go to https://worc.readthedocs.io/en/latest/static/user_manual.html

Some things we would advice to always do:
  - Run actual experiments on the full settings (coarse=False):
  
      ``coarse = False``
      
      ``experiment.binary_classification(coarse=coarse)``
      
  **Note**: this will result in more computation time. We therefore recommmend
  to run this script on either a cluster or high performance PC. If so,
  you may change the execution to use multiple cores to speed up computation
  just before before experiment.execute():
  
      ``experiment.set_multicore_execution()``


  - Add extensive evaluation: experiment.add_evaluation() before experiment.execute():
  
      ``experiment.add_evaluation()``